#### Input Spectrum Data Preprocessing
- Create an output parameter dataset with `obsid` as the key, in preparation for merging with the input spectrum dataset.

- Create an input spectrum dataset, also with `obsid` as the key.
  - LAMOST official website, "It is worth noting that the wavelength range of the flattened spectrum is [3800 Å, 8900 Å]"

- The flux of the flattened spectrum is incorporated into the atmospheric parameter dataset to form the training dataset `LAMOST_Gaia_APOGEE_param.csv`
  - The `flag` field in the file indicates whether there are flux points that have been simply set to zero (possibly bad spectra).


#### 输入光谱数据预处理
- 制作输出参数数据集，以obsid为键，为与输入光谱数据集拼接做准备
- 制作输入光谱数据集，同样以obsid为键
  - LAMOST官网！"值得注意的是：拉平光谱的波长范围是[3800 Å, 8900 Å]"

- 拉平后的光谱流量并入大气参数数据集共同构成训练数据集 `LAMOST_Gaia_APOGEE_param.csv`
  - 文件中 `flag` 字段表示是否存在流量点被简单置零(可能为坏光谱)

In [1]:
from astropy.io import fits
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
import os
plt.style.use('default')

# spectra file path
LAMOST_Gaia_APOGEE_specdir_path = 'data/spec_dir/'
LAMOST_Gaia_APOGEE_path = 'data/LAMOST_Gaia_APOGEE.csv'
# output path
LAMOST_Gaia_APOGEE_parampath = 'data/LAMOST_Gaia_APOGEE_param.csv'

In [2]:
# Fitting a continuous spectrum to the red and blue parts respectively    
# find strong lines
def find_strong_line(wave, flux):
    coeff = np.polyfit(wave, flux, 5)
    formula = np.poly1d(coeff)
    y = formula(wave)
    d = y - flux
    sigma = np.std(d)
    n = np.where((flux < y - 3 * sigma) | (flux > y + 3 * sigma))
    a = n[0]
    i = 0
    left = []
    right = []
    while i < len(a):
        head = i
        length = 0
        while (i < len(a) - 1) and (a[i + 1] == a[i] + 1):
            length += 1
            i += 1
        if head == 0:
            left.append(np.max([a[head] - length, 0]))
        else:
            left.append(np.max([a[head] - length, a[head - 1] + 1]))

        if i == len(a) - 1:
            right.append(np.min([a[i] + length, len(wave) - 1]))
        else:
            right.append(np.min([a[i] + length, a[i + 1] - 1]))
        i += 1
    return wave[left], wave[right]

# mask strong lines
def mask_strong_line(wave, flux, left, right):
    for i in range(len(left)):
        n = np.where((wave < left[i]) | (wave > right[i]))
        wave = wave[n]
        flux = flux[n]
    return [wave, flux]

def find_conti_spec(wave, flux, nploy):
    for i in np.arange(10):
        coeff = np.polyfit(wave, flux, nploy)
        formula = np.poly1d(coeff)
        y = formula(wave)
        d = y - flux
        sigma = np.std(d)
        n = np.where(d < sigma)
        wave = wave[n]
        flux = flux[n]
    return formula

def normalize_blue_spec(wave, flux):
    left, right = find_strong_line(wave, flux)
    wave1, flux1 = mask_strong_line(wave, flux, left, right)
    formula = find_conti_spec(wave1, flux1, 5)
    conti = formula(wave)
    return flux / conti

def normalize_red_spec(wave, flux):
    left, right = find_strong_line(wave, flux)
    wave1, flux1 = mask_strong_line(wave, flux, left, right)
    formula = find_conti_spec(wave1, flux1, 5)
    conti = formula(wave)
    return flux / conti

def get_fileNameList(file_dir):
    # `root` is the current directory path, 
    # `dirs` are all subdirectories in the current path, 
    # `files` are all non-directory files in the current path.
    # root当前目录路径, dirs当前路径下所有子目录, files当前路径下所有非目录子文件
    for root, dirs, files in os.walk(file_dir): 
        fileNameList = files
    return fileNameList

In [3]:
LAMOST_Gaia_APOGEE = pd.read_csv(LAMOST_Gaia_APOGEE_path)

In [4]:
# Create an output parameter dataset using `obsid` as the key, in preparation for merging with the input spectrum dataset.
# 制作输出参数数据集，以obsid为键，为与输入光谱数据集拼接做准备
id = ['obsid']
LAMOST_Gaia_APOGEE_labels = ['TEFF_1','LOGG_1','FE_H','mg_geo','bp_rp_0']

LAMOST_Gaia_APOGEE_param =  LAMOST_Gaia_APOGEE.loc[:,id+LAMOST_Gaia_APOGEE_labels]
LAMOST_Gaia_APOGEE_param.rename(columns = {'TEFF_1':'teff','LOGG_1':'logg','FE_H':'feh'},inplace=True)

In [5]:
LAMOST_Gaia_APOGEE_spec_namelist = get_fileNameList(LAMOST_Gaia_APOGEE_specdir_path)

In [9]:
fileList = LAMOST_Gaia_APOGEE_spec_namelist
filedir = LAMOST_Gaia_APOGEE_specdir_path

flux = []
obsid = []
flag = [] # flag=1 表示有超出阈值的流量点被置1, flag=0 则没有
wave = np.arange(3925,8800,1)
j = 0

for file in fileList:
    try:
        file_path = filedir + file
        hdu = fits.open(file_path,memmap=False)
        id = hdu[0].header['obsid']
        z = hdu[0].header['z']
        bad_pix = np.append(np.nonzero(hdu[1].data[0][3]),np.nonzero(hdu[1].data[0][4]))
        f = np.delete(hdu[1].data[0][0],bad_pix) # 去除坏像素，与标识位&或标识位
        w = np.delete(hdu[1].data[0][2],bad_pix)
        w = w / (1 + z) # rest wave
        # !!!
        f_smooth = savgol_filter(f, 15, 7)
        # 分红蓝端去除连续谱
        index_b = np.where(w < 6000)
        index_r = np.where(w >= 6000)
        w_r = w[index_r]
        w_b = w[index_b]
        f_smooth_b = f_smooth[index_b]
        f_smooth_r = f_smooth[index_r]

        warnings.simplefilter('ignore', np.RankWarning)

        norm_b_smooth = normalize_blue_spec(w_b,f_smooth_b)
        norm_r_smooth = normalize_red_spec(w_r,f_smooth_r)
        norm_smooth = np.append(norm_b_smooth,norm_r_smooth)

        # 拟合
        f_inter_smooth = interp1d(w,norm_smooth)
        # 自己写的拉平函数的流量 插值统一波长范围
        f_normed_smooth = f_inter_smooth(wave)

        # !!!
        f_check = np.array(f_normed_smooth)
        cond = np.where((f_check > 1.2) | (f_check < 0.1))[0]

        if len(cond) == 0:
            flux.append(f_normed_smooth)
            flag.append(0)
        elif len(cond) != 0:
            f_check[cond] = 1
            flux.append(f_check)
            flag.append(1)
            
        obsid.append(id)
        hdu.close()

        j = j + 1
        if j % 1000 == 0:
            print(j)

    # Spectra wavelengths not within the required range
    except Exception as e:
        if file.endswith('.fits'):
            print('An exception occurred. Error File: ' + file)
            print('wavelength: ','[',w[0],',',w[len(w)-1],']')
            print(j)
    continue

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000


In [10]:
# flag=1 indicates that there are flux points exceeding the threshold that have been set to 1, otherwise flag=0
firstline = ['obsid','flag'] + [str(i) for i in list(np.arange(3925,8800,1))]
obsid_pd = pd.DataFrame(obsid)
flag_pd = pd.DataFrame(flag)
flux_pd = pd.DataFrame(flux)
all = pd.concat([obsid_pd,flag_pd,flux_pd],axis=1)
all.columns=firstline

In [11]:
# Save flux into the LAMOST_Gaia_param table (using obsid as the key)
LAMOST_Gaia_APOGEE_paramall = LAMOST_Gaia_APOGEE_param.merge(all,how='left',on='obsid')
LAMOST_Gaia_APOGEE_paramall.to_csv(LAMOST_Gaia_APOGEE_parampath,index=False)